# Introduction to Probabilistic Modeling in Pyro

**This Tutorial is adapted from [https://pyro.ai/examples/intro_part_i.html](https://pyro.ai/examples/intro_part_i.html)*

**Pyro** is a [universal probabilistic programming language (PPL)](https://pyro.ai/) in Python with a PyTorch backend. It enables flexible implementation for deep probabilistic models, which unifies the strengths of deep learning and probabilistic models. As stated in its [website](https://pyro.ai/), Pyro was designed based on the following principles:  

- **Universal**: Pyro can represent *any* computable probability distribution.
- **Scalable**: Pyro scales to *large data sets* with little overhead.
- **Minimal**: Pyro is implemented with a small core of powerful, composable abstractions.
- **Flexible**: Pyro aims for automation and control. 

In this tutorial, we will explore the basic building blocks of Pyro. The following tutorial is an adaptation for the documentation and the examples published in Pyro's [official website](https://pyro.ai/examples/intro_part_i.html). Before we start, we need to import both pyro and torch libraries.

In [2]:
import torch
import pyro

## Stochastic Functions

The basic unit of a probabilistic program is the *stochastic function*. This is a Python callable that combines two ingredients:

- **Deterministic Python code** and
- **primitive stochastic functions** that call a random number generator. 

More concretely, a stochastic function can be *any Python object* with a __call__() method, like a function, a method, or a PyTorch nn.Module. Throughout the tutorials, we will often call *stochastic functions models*, since stochastic functions can be used to represent simplified or abstract descriptions of a process by which data are generated. Expressing models as stochastic functions means that models can be composed, reused, imported, and serialized just like regular Python callables.

### Primitive Stochastic Functions

Primitive stochastic functions (or distributions) are an important class of **stochastic functions** for which we can explicitly compute the *probability of the outputs given the inputs*. We start with an example for using primitive stochastic functions to draw a sample $x$ from the unit normal distribution $\mathcal{N}(0,1)$ as follows:

In [9]:
mean     = 0.   # zero mean 
variance = 1.   # unit variance
normal   = torch.distributions.Normal(mean, variance) # create a normal distribution object

# Note: we could use the wrapper: pyro.distributions.Normal(mean, variance) instead

x = normal.rsample() # draw a sample from N(0,1)

print("sample", x)
print("log prob", normal.log_prob(x))

sample tensor(0.0462)
log prob tensor(-0.9200)


Let us plot the histogram to check if the generated data is indeed Gaussian:

In [ ]:
import seaborn as sns
%matplotlib inline 

import numpy as np

data_points = [normal.rsample().numpy() for k in range(1000)]

sns.distplot(data_points)

Here, **torch.distributions.Normal** is an instance of **Distribution class** that takes parameters and provides sample and score methods. Pyro's distribution library **pyro.distributions** is a thin wrapper around **torch.distributions** meant to make use of PyTorch's fast tensor math and autograd capabilities.

### Building a Simple Model




A probabilistic program is built up by composing primitive stochastic functions and deterministic computation. Since we are ultimately interested in probabilistic programming because we want to model things in the real world, let's start with a model of something concrete.

Let's suppose we have a bunch of data with daily mean temperatures and cloud cover. We want to reason about how temperature interacts with whether it was sunny or cloudy. A simple stochastic function that describes how that data might have been generated is given by:

In [39]:
def weather():
    
    cloudy     = torch.distributions.Bernoulli(0.3).sample()
    cloudy     = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    
    mean_temp  = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    var_temp   = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp       = torch.distributions.Normal(mean_temp, var_temp).rsample()
    
    return cloudy, temp.item()

In [43]:
weather()

('sunny', 95.69055938720703)

As we can see, **weather** is entirely independent of Pyro - it only calls PyTorch. We need to turn it into a Pyro program if we want to use this model for anything other than sampling fake data.

### The pyro.sample Primitive

To turn **weather** into a Pyro program, we'll replace the **torch.distributions** with **pyro.distributions** and the **.sample()** and **.rsample()** calls with calls to **pyro.sample**, one of the core language primitives in Pyro. Using **pyro.sample** is as simple as calling a primitive stochastic function with one important difference:

In [45]:
x = pyro.sample("my_sample", pyro.distributions.Normal(mean, variance))
x

tensor(-0.2579)

Just like a direct call to **torch.distributions.Normal().rsample()**, this returns a sample from the unit normal distribution.** The crucial difference is that this sample is named.** Pyro's backend uses these names to uniquely identify sample statements and change their behavior at runtime depending on how the enclosing stochastic function is being used. *As we will see, this is how Pyro can implement the various manipulations that underlie inference algorithms.*

Now that we've introduced **pyro.sample** and **pyro.distributions** we can rewrite our simple model as a Pyro program:

In [48]:
def weather():
    
    cloudy  = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy  = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    
    mean_temp     = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    variance_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, variance_temp))
    
    return cloudy, temp.item()

for _ in range(3):
    
    print(weather())

('cloudy', 58.179054260253906)
('sunny', 85.4627456665039)
('sunny', 59.515235900878906)


Here, **weather()** is still a non-deterministic Python callable that returns two random samples. Because the randomness is now invoked with **pyro.sample**, however, it is much more than that. **In particular weather() specifies a joint probability distribution over two named random variables: cloudy and temp.** As such, it defines a probabilistic model that we can reason about using the techniques of probability theory. For example we might ask: if I observe a temperature of 70 degrees, how likely is it to be cloudy? 

### Stochastic Recursion, Higher-order Stochastic Functions, and Random Control Flow

We have now seen how to define a simple model. Building off of it is easy. For example:

In [50]:
def ice_cream_sales():
    
    cloudy, temp = weather()
    
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream      = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    
    return ice_cream

In [118]:
ice_cream_sales()

tensor(38.8044)

This kind of modularity, familiar to any programmer, is obviously very powerful. But is it powerful enough to encompass all the different kinds of models we'd like to express?

It turns out that because Pyro is embedded in Python, stochastic functions can contain arbitrarily complex deterministic Python and randomness can freely affect control flow. For example, we can construct recursive functions that terminate their recursion nondeterministically, provided we take care to pass **pyro.sample** unique sample names whenever it's called. For example we can define a geometric distribution that counts the number of failures until the first success like so:

In [56]:
def geometric(p, t=None):
    
    if t is None:
        
        t = 0
    
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)


In [187]:
geometric(0.01)

92

Note that the names $x_0$, $x_1$, etc., in **geometric()** are generated dynamically and that different executions can have different numbers of named random variables. We are also free to define stochastic functions that accept as input or produce as output other stochastic functions:

In [61]:
def normal_product(mean, variance):
    
    z1 = pyro.sample("z1", pyro.distributions.Normal(mean, variance))
    z2 = pyro.sample("z2", pyro.distributions.Normal(mean, variance))
    
    y  = z1 * z2
    
    return y

def make_normal_normal():
    
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn        = lambda scale: normal_product(mu_latent, scale)
    
    return fn

print(make_normal_normal()(1.))

tensor(2.3250)


Here make_normal_normal() is a stochastic function that takes one argument and which, upon execution, generates three named random variables.

### Building a Markov Chain

In a final example, we build a variable-length Markov chain. Similar to the previous examples, here we dynamically generate a different number of random variables in different executions. 

In [191]:
num_states      = 3
average_seq_len = 100

P_t        = torch.tensor([[0.7, 0.2, 0.1], 
                           [0.3, 0.6, 0.1], 
                           [0.3, 0.2, 0.5]])

P_0        = torch.tensor([0.3, 0.4, 0.3])


def geometric_(p):
    
    seq_length = pyro.sample("Seq_len", pyro.distributions.Bernoulli(p))
    
    if seq_length.item() == 1:
        return 0
    else:
        return 1 + geometric(p)


def transition(P_t, z_t, t):
    
    state_space   = list(range(num_states))    
    state_space   = torch.tensor([np.float(state_space[k]) for k in range(len(state_space))])
    
    transition_probabilities = P_t[int(z_t),:]
    
    z = pyro.sample("z_{}".format(t), pyro.distributions.Multinomial(total_count=1, probs=transition_probabilities))
    
    return (state_space*z).sum().numpy()


def Markov_chain(P_0, P_t, num_states, average_seq_len):
    
    Seq_len_      = geometric_(1/average_seq_len)
    
    state_space   = list(range(num_states))    
    state_space   = torch.tensor([np.float(state_space[k]) for k in range(len(state_space))])
    
    initial_state = pyro.sample("z_0", pyro.distributions.Multinomial(total_count=1, probs=P_0)).sum().numpy()
    
    states = [initial_state]
    
    for t in range(Seq_len_):

        states.append(transition(P_t, states[-1], t + 1))
    
    return states
    

In [192]:
Markov_chain(P_0, P_t, num_states, average_seq_len)

[array(1., dtype=float32),
 array(1., dtype=float32),
 array(1., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(1., dtype=float32),
 array(2., dtype=float32),
 array(1., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(1., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(2., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 array(1., dtype=float32),
 array(1., dtype=float32),
 array(1., dtype=float32),
 array(1., dtype=float32),
 array(0., dtype=float32),
 array(0., dtype=float32),
 

We will be using a similar stochastic model to model time series data through Hidden Markov models in the following tutorials...